# D1 - Walvisualisatie
Noor Koevoet, Glenn de Graaff, Anne Beks, Job Kolhorn

D1

## Introductie
Persvrijheid is een principe dat stelt dat de pers vrij is om te schrijven, zeggen en publiceren wat zij willen. De overheid is in beginsel verboden om censuur toe te passen en mag niet vooraf controleren wat wel of niet gezegd mag worden. In Nederland is persvrijheid, samen met het recht op meningsuiting, vastgelegd in artikel 7 van de Grondwet. Ook in andere landen en binnen Europees verband zijn er wetten die persvrijheid en vrijheid van meningsuiting beschermen.

Om te verzekeren dat journalisten hun werk veilig kunnen doen, hebben zij een beschermde status van het Europese Hof voor de Rechten van de Mens (EHRM). Journalisten brengen informatie naar buiten over algemeen belangrijke zaken en zijn daarom van cruciaal belang voor de democratie. Hoewel er vrijheid is om te publiceren wat men wil, betekent dit niet dat dit onbestraft kan gebeuren. Wanneer publicaties discriminerend zijn of aanzetten tot opruiing, kan iemand aangeklaagd en veroordeeld worden. Dit gebeurt echter pas achteraf; vóór publicatie is censuur niet toegestaan.

In veel landen is persvrijheid echter niet gegarandeerd. In landen met een autocratisch regime is de vrijheid van meningsuiting vaak beperkt of volledig afwezig. Ook in landen waar conflicten gaande zijn, zijn er mogelijk meerdere machten aan de leiding, of lijkt er niemand aan de macht te zijn. In zulke situaties is het moeilijk om te waarborgen dat de wet wordt nageleefd.

Deze data-analyse zal onderzoeken of de mate van persvrijheid wordt beïnvloed door conflicten of de mate van democratie. Door gebruik te maken van data van Reporters Without Borders (RSF), The Armed Conflict Location & Event Data Project (ACLED) en Our World in Data, zullen we scores en ranglijsten analyseren rondom persvrijheid, democratie en conflicten wereldwijd. Het perspectief dat conflicten de persvrijheid verminderen en het perspectief dat autocratische bestuursvormen de persvrijheid verminderen, worden hier onderzocht.

[Mensenrechten.nl](https://www.mensenrechten.nl/themas/vrijheid-van-meningsuiting)


## Dataset and Preprocessing

Tekst

In [93]:
import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
import plotly.express as px

import ipywidgets as widgets
from ipywidgets import interact

import numpy as np

from IPython.display import display

init_notebook_mode(connected=True)

In [ ]:
# Noor's data

conflict_data = pd.read_excel('./data/01-2024-conflic-index.xlsx')
data_conflict_ranking = conflict_data[['country', 'Index Ranking 2023']]

data_pers = pd.read_csv('./data/2023.csv', sep=';')
data_pers.rename(columns = {'Country_EN':'country'}, inplace=True)

data_pers_ranking = data_pers[['country', 'Rank']]

data_both = pd.merge(data_pers_ranking, data_conflict_ranking, on='country', how='inner')

def determine_color(rank):
    if rank <= 10:
        return 'red'
    elif rank <= 30:
        return 'orange'
    elif rank <= 50:
        return 'yellow'
    else:
        return 'blue'

df = pd.DataFrame(data_both)
df['Color'] = df['Index Ranking 2023'].apply(determine_color)


### Data voor visualisatie 1 & 2

In [76]:
# create a dataframe for the pressfreedom data of every year, renamed a few columns from the latest three datasets so they are the same as the rest
df_24 = pd.read_csv('./data/persvrijheid/2024.csv', sep = ';').rename(columns={"Score": "Score N", "Rank": "Rank N", "Country_FR": "FR_country", "Country_EN": "EN_country", "Country_ES": "ES_country", "Country_FA": "FA_country", "Country_AR": "AR_country"})

df_23 = pd.read_csv('./data/persvrijheid/2023.csv', sep = ';').rename(columns={"Score": "Score N", "Rank": "Rank N", "Country_FR": "FR_country", "Country_EN": "EN_country", "Country_ES": "ES_country", "Country_FA": "FA_country", "Country_AR": "AR_country"})

df_22 = pd.read_csv('./data/persvrijheid/2022.csv', sep = ';').rename(columns={"Score": "Score N", "Rank": "Rank N", "Country_FR": "FR_country", "Country_EN": "EN_country", "Country_ES": "ES_country", "Country_FA": "FA_country", "Country_AR": "AR_country"})

df_21 = pd.read_csv('./data/persvrijheid/2021.csv', sep = ';')
df_20 = pd.read_csv('./data/persvrijheid/2020.csv', sep = ';')
df_19 = pd.read_csv('./data/persvrijheid/2019.csv', sep = ';')

In [77]:
# combineer de data voor elk land vanaf het jaar 2019

df_combined_19 = pd.concat([df_24, df_23, df_22, df_21, df_20, df_19])
df_grouped_19 = df_combined_19.groupby(by='ISO')

Democratie data (2023) voorbereiden:

In [78]:
# maak de data klaar voor de persvrijheid en democratie scores van 2023

data_23 = pd.read_csv('./data/persvrijheid/2023.csv', sep = ';')
data_dem = pd.read_csv('./data/democracy.csv')
data_dem_23 = data_dem[data_dem["Year"]==2023].sort_values(by="Democracy score", ascending=False).reset_index().rename(columns={'Entity':'Country_EN'})

In [79]:
# geef de landen een ranking gebaseerd om de democratie score

score = data_dem_23.iloc[0]["Democracy score"]
rank = 1
ranking = []
for country in data_dem_23.index:
    count = country + 1
    if len(ranking)==0:
        ranking.append(1)
    else: 
        if data_dem_23.iloc[country]["Democracy score"] == data_dem_23.iloc[country-1]["Democracy score"]:
            ranking.append(ranking[-1])
        elif data_dem_23.iloc[country]["Democracy score"] < data_dem_23.iloc[country-1]["Democracy score"]:
            ranking.append(count)
            
data_dem_23["ranking"] = ranking

In [80]:
#join de democratie en persvrijheid data tot één dataframe
data_joined = pd.merge(data_dem_23, data_23, on='Country_EN', how='inner')

### Data voor visualisatie 3

Overige democratie data voorbereiden (vanaf 2019)

In [81]:
# maak ook de democratie data klaar voor de andere jaartallen vanaf 2019 (voor 2024 is er geen data)

data_dem_22 = data_dem[data_dem["Year"]==2022].sort_values(by="Democracy score", ascending=False).reset_index().rename(columns={'Entity':'Country_EN'})
data_dem_21 = data_dem[data_dem["Year"]==2021].sort_values(by="Democracy score", ascending=False).reset_index().rename(columns={'Entity':'Country_EN'})
data_dem_20 = data_dem[data_dem["Year"]==2020].sort_values(by="Democracy score", ascending=False).reset_index().rename(columns={'Entity':'Country_EN'})
data_dem_19 = data_dem[data_dem["Year"]==2019].sort_values(by="Democracy score", ascending=False).reset_index().rename(columns={'Entity':'Country_EN'})



# 2022
score = data_dem_22.iloc[0]["Democracy score"]
rank = 1
ranking = []
for country in data_dem_22.index:
    count = country + 1
    if len(ranking)==0:
        ranking.append(1)
    else: 
        if data_dem_22.iloc[country]["Democracy score"] == data_dem_22.iloc[country-1]["Democracy score"]:
            ranking.append(ranking[-1])
        elif data_dem_22.iloc[country]["Democracy score"] < data_dem_22.iloc[country-1]["Democracy score"]:
            ranking.append(count)
            
data_dem_22["ranking"] = ranking


#2021
score = data_dem_21.iloc[0]["Democracy score"]
rank = 1
ranking = []
for country in data_dem_21.index:
    count = country + 1
    if len(ranking)==0:
        ranking.append(1)
    else: 
        if data_dem_21.iloc[country]["Democracy score"] == data_dem_21.iloc[country-1]["Democracy score"]:
            ranking.append(ranking[-1])
        elif data_dem_21.iloc[country]["Democracy score"] < data_dem_21.iloc[country-1]["Democracy score"]:
            ranking.append(count)
            
data_dem_21["ranking"] = ranking


#2020
score = data_dem_20.iloc[0]["Democracy score"]
rank = 1
ranking = []
for country in data_dem_20.index:
    count = country + 1
    if len(ranking)==0:
        ranking.append(1)
    else: 
        if data_dem_20.iloc[country]["Democracy score"] == data_dem_20.iloc[country-1]["Democracy score"]:
            ranking.append(ranking[-1])
        elif data_dem_20.iloc[country]["Democracy score"] < data_dem_20.iloc[country-1]["Democracy score"]:
            ranking.append(count)
            
data_dem_20["ranking"] = ranking


#2019
score = data_dem_19.iloc[0]["Democracy score"]
rank = 1
ranking = []
for country in data_dem_19.index:
    count = country + 1
    if len(ranking)==0:
        ranking.append(1)
    else: 
        if data_dem_19.iloc[country]["Democracy score"] == data_dem_19.iloc[country-1]["Democracy score"]:
            ranking.append(ranking[-1])
        elif data_dem_19.iloc[country]["Democracy score"] < data_dem_19.iloc[country-1]["Democracy score"]:
            ranking.append(count)
            
data_dem_19["ranking"] = ranking




# combine them all
data_dem_combined = pd.concat([data_dem_23, data_dem_22, data_dem_21, data_dem_20, data_dem_19])

### Data voor visualisatie 4

Conflict data voorbereiden

In [82]:
data_conflict = pd.read_csv('./data/conflict.csv').rename(columns={'country': 'Country_EN'})

In [83]:
# join de data

data_joined_2 = pd.merge(data_dem_23, data_conflict, on='Country_EN', how='inner')


## Conflicten Verminderen de Persvrijheid

### Argument 1

**Landen die in conflict zijn hebben op dat moment een lage persvrijheid.**

Het is algemeen bekend dat conflicten een negatieve impact hebben op verschillende aspecten van de samenleving, hier is persvrijheid ook een deel van. Tijdens conflicten wordt de veiligheid van journalisten vaak in gevaar gebracht, waardoor ze hun werk niet vrij kunnen uitvoeren. De dreiging van geweld en intimidatie zorgt ervoor dat journalisten zichzelf censureren of stoppen met rapporteren over gevoelige onderwerpen. De pie chart hieronder biedt een helder inzicht in de verdeling van conflictniveaus wereldwijd. Uit het diagram blijkt dat 71,4% van de landen momenteel een lage of inactieve conflictstatus heeft. Deze landen ervaren doorgaans minder ernstige beperkingen op persvrijheid. In tegenstelling hiermee bevindt de overige 28,6% van de landen zich in een hoge conflictstatus, waar persvrijheid mogelijk wordt ingeperkt. Dit betekent dat bijna een derde van de journalisten in deze landen te maken heeft met aanzienlijke uitdagingen en belemmeringen bij hun werk.

In [96]:
df['Color'] = df['Rank'].apply(determine_color)

color_counts = df['Color'].value_counts()

color_labels = {
    'red': 'Extreem',
    'orange': 'Hoog',
    'yellow': 'Turbulent',
    'blue': 'laag/inactief'
}


labels = [color_labels[color] for color in color_counts.index]


fig = go.Figure(data=[go.Pie(
    labels = labels,
    values = color_counts.values,
    marker = dict(colors = color_counts.index)
)])

fig.update_layout(
    title = 'Aantal landen met bepaalde conflict status (index level) ',
    legend_title = 'Index level'
)

fig.show()

In [97]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df['Index Ranking 2023'],
    y=df['Rank'],
    mode='markers',
    text=df['country'],
    marker=dict(
        color=df['Color'],
        size=10
    ),
    name='Land',
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=[None], y=[None],
    mode='markers',
    marker=dict(color='red', size=10),
    legendgroup='group1',
    showlegend=True,
    name='Extreem (Top 10)'
))
fig.add_trace(go.Scatter(
    x=[None], y=[None],
    mode='markers',
    marker=dict(color='orange', size=10),
    legendgroup='group2',
    showlegend=True,
    name='Hoog (11-30)'
))
fig.add_trace(go.Scatter(
    x=[None], y=[None],
    mode='markers',
    marker=dict(color='yellow', size=10),
    legendgroup='group3',
    showlegend=True,
    name='Turbulent (31-50)'
))
fig.add_trace(go.Scatter(
    x=[None], y=[None],
    mode='markers',
    marker=dict(color='blue', size=10),
    legendgroup='group4',
    showlegend=True,
    name='Laag/inactief (>50)'
))

fig.add_trace(go.Scatter(
    x=[min(df['Index Ranking 2023']), max(df['Index Ranking 2023'])],
    y=[max(df['Rank']), min(df['Rank'])],
    mode='lines',
    line=dict(color='black'),
    name='Trend lijn'
))

fig.update_layout(
    title='Press Freedom Index Ranking vs Conflict Index Ranking (2023)',
    xaxis_title='Conflict Index Ranking (Index Ranking 2023)',
    yaxis_title='Press Freedom Index Ranking (Rank)',
    showlegend=True,
    height=700
)

fig.show()

correlation = df['Index Ranking 2023'].corr(df['Rank'])

print(f"Correlatie tussen de conflict index ranking en de persvrijheid index ranking: {correlation}")

Correlatie tussen de conflict index ranking en de persvrijheid index ranking: -0.42573893484944786


De analyse van de dataset suggereert dat conflicten een negatieve impact hebben op de persvrijheid. We hebben een significante negatieve correlatie gevonden tussen de Conflict Index Ranking en de Press Freedom Index Ranking voor het jaar 2023. Specifiek toont de visualisatie een correlatie van `-0.43` tussen deze twee indexen. Dit betekent dat naarmate een land hoger scoort op de Conflict Index Ranking (wat wijst op een lagere mate van conflict), het doorgaans lager scoort op de Press Freedom Index Ranking (wat wijst op meer persvrijheid).


### Argument 2

**Naarmate een conflict escaleert, verslechtert de persvrijheid nog meer.**

De relatie tussen conflicten en persvrijheid is complex en vertoont vaak zorgwekkende trends. Wanneer conflicten escaleren, lijkt de persvrijheid in samenlevingen significant te verslechteren. Dit fenomeen wordt goed geïllustreerd in gebieden zoals Palestina en Syrië. Gedurende de periode van 2019 tot 2023 is de Conflict Index Ranking voor Syrië relatief stabiel gebleven, in contrast hiermee heeft Palestina een opmerkelijke stijging in de Conflict Index Ranking doorgemaakt, van de 23e plaats in 2019 naar de 3e plaats in 2023, wat duidt op een aanzienlijke escalatie van het conflict in dat gebied.


In [99]:
data_pers_ranking_19 = pd.read_csv('./data/2019.csv', sep=';')
data_pers_ranking_23 = pd.read_csv('./data/2023.csv', sep=';')

data_conflict = pd.read_excel('./data/01-2024-conflic-index.xlsx')

data_pers_ranking_19.rename(columns={'EN_country': 'country', 'Rank N': 'Rank_2019'}, inplace=True)
data_pers_ranking_23.rename(columns={'Country_EN': 'country', 'Rank': 'Rank_2023'}, inplace=True)
data_conflict.rename(columns={'Country': 'country', 'Index Ranking 2023': 'Conflict_Rank_2023', 'Change in ranking since 2019': 'Change_Conflict'}, inplace=True)

data_pers_combined = pd.merge(data_pers_ranking_19[['country', 'Rank_2019']], 
                              data_pers_ranking_23[['country', 'Rank_2023']], 
                              on='country', 
                              how='inner')


data_conflict['Conflict_Rank_2019'] = data_conflict['Conflict_Rank_2023'] + data_conflict['Change_Conflict']


def filter_countries(text, dataset):
    if dataset == 'Conflict':
        return [country for country in data_conflict['country'] if text.lower() in country.lower()]
    else:
        return [country for country in data_pers_combined['country'] if text.lower() in country.lower()]


def plot_countries_ranking(countries):
    if not countries:
        print("Selecteer minstens een land")
        return
    
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=('Conflict Index', 'Press Freedom Index'))
    
    for country in countries:

        selected_data_conflict = data_conflict[data_conflict['country'] == country]
        
        if len(selected_data_conflict) > 0:
            ranking_2019 = selected_data_conflict['Conflict_Rank_2019'].values[0]
            ranking_2023 = selected_data_conflict['Conflict_Rank_2023'].values[0]
            change = ranking_2023 - ranking_2019
            
            if change == 0:
                color = 'blue'
            elif change > 0:
                color = 'green'
            else:
                color = 'red' 
            
            fig.add_trace(go.Scatter(
                x = ['2019', '2023'],
                y = [ranking_2019, ranking_2023],
                mode = 'lines+markers',
                marker = dict(color=color),
                line = dict(color=color),
                name = f'{country} Conflict ({ranking_2019} -> {ranking_2023})'
            ), row=1, col=1)
        
        selected_data_pers = data_pers_combined[data_pers_combined['country'] == country]
        
        if len(selected_data_pers) > 0:
            ranking_2019 = selected_data_pers['Rank_2019'].values[0]
            ranking_2023 = selected_data_pers['Rank_2023'].values[0]
            change = ranking_2023 - ranking_2019
            
            if change == 0:
                color = 'blue'
            elif change > 0:
                color = 'red'
            else:
                color = 'green' 
            
            fig.add_trace(go.Scatter(
                x = ['2019', '2023'],
                y = [ranking_2019, ranking_2023],
                mode = 'lines+markers',
                marker = dict(color=color),
                line = dict(color=color),
                name = f'{country} Press Freedom ({ranking_2019} -> {ranking_2023})'
            ), row=2, col=1)
        
    if len(countries) == 1:
        title = f'Verandering in Ranking voor {countries[0]}'
    elif len(countries) == 2:
        title = f'Verandering in Ranking voor {" en ".join(countries)}'
    elif len(countries) > 5:
        title = f'Verandering in Ranking voor {len(countries)} landen'
    else:
        title = f'Verandering in Ranking voor {", ".join(countries)}'
        
    fig.update_layout(
        height=800,
        title_text=title,
        showlegend=True,
    )
    
    fig.update_xaxes(title_text='Jaar', row=1, col=1)
    fig.update_xaxes(title_text='Jaar', row=2, col=1)
    fig.update_yaxes(title_text='Conflict Index Ranking', row=1, col=1)
    fig.update_yaxes(title_text='Press Freedom Index Ranking', row=2, col=1)
    
    fig.show()

dataset_dropdown = widgets.Dropdown(
    options=['Conflict', 'Press Freedom'],
    value='Conflict',
    description='Dataset:'
)

country_dropdown = widgets.SelectMultiple(
    options = data_conflict['country'].unique().tolist() if dataset_dropdown.value == 'Conflict' else data_pers_combined['country'].unique().tolist(),
    description = 'Landen:',
    disabled = False,
    layout = widgets.Layout(height='200px', width='400px'),
)

search_text = widgets.Text(
    value = '',
    placeholder = 'Zoek landen',
    description = 'Zoeken:',
    disabled = False
)

def handle_search(change):
    search_results = filter_countries(change.new, dataset_dropdown.value)
    
    selected_countries = list(country_dropdown.value)
    country_dropdown.options = search_results
    
    country_dropdown.value = [country for country in selected_countries if country in search_results]

def update_country_dropdown(change):
    search_results = filter_countries(search_text.value, change.new)
    country_dropdown.options = search_results
    country_dropdown.value = []

search_text.observe(handle_search, names='value')
dataset_dropdown.observe(update_country_dropdown, names='value')

display(dataset_dropdown)
display(search_text)

interact(plot_countries_ranking, countries=country_dropdown)

Dropdown(description='Dataset:', options=('Conflict', 'Press Freedom'), value='Conflict')

Text(value='', description='Zoeken:', placeholder='Zoek landen')

interactive(children=(SelectMultiple(description='Landen:', layout=Layout(height='200px', width='400px'), opti…

<function __main__.plot_countries_ranking(countries)>

De visualisatie toont deze trends duidelijk. Het beeld schetst de mogelijke negatieve correlatie tussen de Conflict Index en de Press Freedom Index, waarbij landen met hogere conflictniveaus vaak lagere persvrijheidsscores vertonen. Dit suggereert dat journalisten in deze regio's te maken hebben met verhoogde risico's zoals geweld, intimidatie en censuur, wat hun mogelijkheid om vrij te rapporteren beperkt.

De data wijst erop dat tijdens conflicten vaak noodmaatregelen en censuur worden ingevoerd om de berichtgeving te controleren en de publieke opinie te beïnvloeden, wat verdere inperkingen van de persvrijheid met zich meebrengt. Hoewel deze correlatie aangeeft hoe conflicten de persvrijheid kunnen beïnvloeden, moeten we erkennen dat andere factoren zoals autoritaire regimes en economische instabiliteit ook een rol kunnen spelen. Desalniettemin benadrukt deze analyse het cruciale belang van het beschermen van journalisten en het handhaven van vrije media, vooral in tijden van crisis.


## Autocratische Bestuursvormen Verminderen de Persvrijheid

### Argument 1

**Landen met een meer autocratische bestuursvorm hebben een lagere persvrijheid.**

Het verband tussen bestuursvormen en persvrijheid is cruciaal binnen de context van democratie en mensenrechten. Autocratische regimes, die worden gekenmerkt door een sterke concentratie van macht en beperkte politieke vrijheden, vertonen vaak tendensen van onderdrukking van vrije media. Deze onderdrukking dient niet alleen om de macht te versterken, maar ook om elke vorm van oppositie streng te controleren en te beperken. Het resultaat hiervan is vaak censuur, arrestaties van journalisten en een strikte controle over mediakanalen, wat kan leiden tot lagere persvrijheidsscores in deze landen.

In [100]:
# creeer een scatterplot om de relatie tussen de democratie en persvrijheid aan te geven per land

fig = px.scatter(data_joined, x='ranking', y='Rank', title = 'Democracy index ranking vs Press freedom index ranking 2023 ', hover_data=['Country_EN'], trendline="ols")

fig.update_layout(
    xaxis_title="Democracy index ranking 2023",
    yaxis_title="Press freedom index ranking 2023"
)

for trace in fig.data:
    if trace.mode == "lines":
        trace.line.color = "black"

fig.show()

De visualisatie bestaat uit scatterplots die de relatie tussen de Democracy Index en de Press Freedom Index per land weergeven. Deze grafiek biedt een visuele representatie van de dynamiek tussen bestuursvorm en persvrijheid. Over het algemeen tonen landen met hogere scores op de Democracy Index, wat duidt op meer autocratische bestuursvormen, ook hogere scores op de Press Freedom Index, wat duidt op minder persvrijheid. Deze scatter plots demonstreren hoe de mate van democratie in een land samenhangt met de mate van persvrijheid die journalisten en mediaorganisaties daar ervaren.

### Argument 2

**Landen met een meer autocratische bestuursvorm hebben vaker conflicten.**

De twee pie charts hieronder bieden inzicht in de veranderingen van de Conflict Index Rankings van 2019 tot 2023. Uit de eerste pie chart blijkt dat `28%` van de landen hun Conflict Index Ranking heeft zien verslechteren gedurende deze periode. Tegelijkertijd toont de tweede pie chart dat in dezelfde periode `46.7%` van de landen een verslechtering heeft ervaren. Deze overlappende groep van landen, waarin zowel de Conflict Index Rankings als de persvrijheid mogelijk zijn verslechterd, wijst op een zorgwekkende trend van toenemende conflicten en mogelijke beperkingen van persvrijheid.

In [101]:
df = pd.DataFrame(data_conflict)

data_conflict['Conflict_Rank_2019'] = data_conflict['Conflict_Rank_2023'] + data_conflict['Change_Conflict']

conditions = [
    (df['Change_Conflict'] > 0),
    (df['Change_Conflict'] < 0),
    (df['Change_Conflict'] == 0)
]
choices = ['Verslechterd', 'Verbeterd', 'Gelijk gebleven']
df['Status'] = np.select(conditions, choices, default='Niet gedefinieerd')

status_counts = df['Status'].value_counts()

fig_status = go.Figure(data=[go.Pie(
    labels=status_counts.index,
    values=status_counts.values,
    marker=dict(colors=['green', 'red', 'blue'])
)])

fig_status.update_layout(
    title='Verandering in Conflict Index Rankings sinds 2019 per 2023',
    legend_title='Status'
)

fig_status.show()

In [102]:
data_pers_combined['Change'] = data_pers_combined['Rank_2023'] - data_pers_combined['Rank_2019']

conditions = [
    (data_pers_combined['Change'] > 0),
    (data_pers_combined['Change'] < 0),
    (data_pers_combined['Change'] == 0)
]
choices = ['Verbeterd', 'Verslechterd', 'Gelijk gebleven']
data_pers_combined['Status'] = np.select(conditions, choices, default='Niet gedefinieerd')

status_counts = data_pers_combined['Status'].value_counts()

fig_status = go.Figure(data=[go.Pie(
    labels=status_counts.index,
    values=status_counts.values,
    marker=dict(colors=['green', 'red', 'blue'])
)])

fig_status.update_layout(
    title='Verandering in Persvrijheidsindex Rankings sinds 2019 per 2023',
    legend_title='Status'
)

fig_status.show()

De scatterplot illustreert verder dat landen met meer autocratische bestuursvormen, gekenmerkt door hogere scores op de Democracy Index Ranking, doorgaans lagere scores hebben op de Conflict Index Ranking. Dit betekent dat autocratische regimes vaak een grotere neiging hebben tot conflicten, mogelijk als gevolg van beperkte politieke participatie, onderdrukking van oppositie en machtsmisbruik. Deze correlatie benadrukt de uitdagingen waarmee autocratische samenlevingen worden geconfronteerd in termen van zowel interne stabiliteit als externe betrekkingen.


## Visualisatie 2

Creeer een line chart van de persvrijheid rankings per jaar. Gebruik ctrl+klik om meerdere landen te selecteren

You can also plot the data using static visualizations, such as the [seaborn](https://seaborn.pydata.org/#) library.

In [90]:
# Persvrijheid scores -  ctrl + klik om meerdere landen te selecteren

# Create a dropdown for country selection
country_dropdown = widgets.SelectMultiple(
    options= df_combined_19['EN_country'].unique(),
    value=[df_combined_19['EN_country'].unique()[0]],
    description='Country',
    disabled=False
)

# Function to update the line chart based on selected countries
def update_chart(selected_countries):
    filtered_df = df_combined_19[df_combined_19['EN_country'].isin(selected_countries)]
    fig = px.line(filtered_df, x='Year (N)', y='Rank N', color='EN_country', 
                  labels={'Democracy_score': 'Democracy Score'}, title='Persvrijheid ranking per land')
    fig.update_layout(
        xaxis_title="Year",
        yaxis_title="Persvrijheid rank"
    )
    fig.show()

# Create an interactive output
output = widgets.interactive_output(update_chart, {'selected_countries': country_dropdown})

# Display the dropdown and the output
display(country_dropdown, output)


SelectMultiple(description='Country', index=(0,), options=('Norway', 'Denmark', 'Sweden', 'Netherlands', 'Finl…

Output()

## Visualisatie 3

Creeer een line chart van de democratie rankings per jaar. Gebruik ctrl+klik om meerdere landen te selecteren.

In [91]:
# democratie scores - ctrl + klik om meerdere landen te selecteren

# Create a dropdown for country selection
country_dropdown = widgets.SelectMultiple(
    options= data_dem_combined['Country_EN'].unique(),
    value=[data_dem_combined['Country_EN'].unique()[0]],
    description='Country',
    disabled=False
)

# Function to update the line chart based on selected countries
def update_chart(selected_countries):
    filtered_df = data_dem_combined[data_dem_combined['Country_EN'].isin(selected_countries)]
    fig = px.line(filtered_df, x='Year', y='ranking', color='Country_EN', 
                  labels={'Democracy_score': 'Democracy Score'}, title='Democratie ranking per land')
    fig.update_layout(
        xaxis_title="Year",
        yaxis_title="Democratie rank"
    )
    fig.show()

# Create an interactive output
output = widgets.interactive_output(update_chart, {'selected_countries': country_dropdown})

# Display the dropdown and the output
display(country_dropdown, output)

SelectMultiple(description='Country', index=(0,), options=('Norway', 'New Zealand', 'Iceland', 'Sweden', 'Finl…

Output()

## Visualisatie 4

In [92]:
# creeer een scatterplot om de relatie tussen de democratie en conflict aan te geven per land

fig = px.scatter(data_joined_2, x='ranking', y='Index Ranking 2023', title = 'Democracy index ranking vs Conflict index ranking 2023', hover_data=['Country_EN'], trendline="ols")

fig.update_layout(
    xaxis_title="Democracy index ranking 2023",
    yaxis_title="Conflict index ranking 2023"
)

for trace in fig.data:
    if trace.mode == "lines":
        trace.line.color = "black"


fig.show()

## Ter verduidelijking:


-----
- 'Conflict index':

Een hogere ranking (bijv. 1) geeft aan dat er in een land veel/grote conflicten zijn
Een lagere ranking (bijv. 150) geeft aan dat er in een land weinig/kleine conflicten zijn

--------
- 'Democratie index':

Een hogere ranking (bijv. 1) geeft aan dat er in een land een heersende democratie is
Een lagere ranking (bijv. 150) geeft aan dat er in een land geen democratie is

---------
- 'Persvrijheid  index':

Een hogere ranking (bijv. 1) geeft aan dat de pers in een land veel vrijheid heeft
Een lagere ranking (bijv. 150) geeft aan dat de pers in een land weinig vrijheid heeft.

## Reflectie

Tekst

## Work distribution

Tekst

## Referenties

Tekst